In [1]:
import geopandas as gpd
import pandas as pd
import geowrangler.vector_zonal_stats as vzs
import geowrangler.raster_zonal_stats as rzs
import geowrangler.area_zonal_stats as azs
import geowrangler.spatialjoin_highest_intersection as shi
from geowrangler import grids
import numpy as np
import folium

import os
import glob
from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Processing generated Isochrone contours

When visualized, the outputed isochrone contours overlap with one another over an area. In this notebook, we aim to assign one travel time contour for each health facility type over an area by getting the minimum travel time. Afterwards, we compute for the population reached in a barangay.

### Input
- Isochrone contour GeoJSON files
- Administrative Boundaries
- WorldPop 2020 Raster

### Output
- Barangay-level dataframe with columns as the travel time and the values are the population reached (sum and percentage) of the narest health facility

## Set-up parameters and directories

In [2]:
PROCESSED_DIR = Path("../../../data/03-processed/")
OUTPUT_DIR = Path("../../../data/04-output/")
GIS_DIR = Path("../../../data/05-gis/")

HOSPITALS_FPATH = Path("../../../data/02-raw/philippines_healthfacilities.gpkg")
ISOCHRONES_DIR = PROCESSED_DIR / "isochrones"
ADMIN_BOUNDS = Path("../../../data/01-admin-bounds/target_admin_bounds.shp")
WP_2020_RASTER = Path("../../../data/02-raw/worldpop/population_count/phl_ppp_2020.tif")

In [3]:
ZOOM_LEVEL = 18  # ~150m
LOCAL_CRS = "EPSG:3857"
PROJ_CRS = "EPSG:4326"

## Load health facilities points

In [4]:
doh_gdf = gpd.read_file(HOSPITALS_FPATH, driver="GPKG")
doh_gdf.head(2)

,id,facilityco,healthfaci,typeofheal,barangay,municipali,province,region,status,address,style,geometry
0,1.0,DOH000000000002277,Calvario Barangay Health Station,Barangay Health Station,Calvario,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.98987 6.65182)
1,2.0,DOH000000000010319,Cabunbata Barangay Health Station,Barangay Health Station,Cabunbata,City Of Isabela,City Of Isabela (not A Province),REGION IX (ZAMBOANGA PENINSULA),None,None,Barangay Health Station,POINT (121.96630 6.67152)


## Load Isochrones Contours

In [5]:
hospital_files = glob.glob(f"{ISOCHRONES_DIR}/iso_hospital*.geojson")
brgy_healthcenter_files = glob.glob(f"{ISOCHRONES_DIR}/iso_brgy_healthcenters*.geojson")
rhu_files = glob.glob(f"{ISOCHRONES_DIR}/iso_rhu*.geojson")

In [6]:
def concat_health_contours(iso_files):
    """
    Concatenate all the contours for each health facility type
    into one dataframe.
    """
    iso_gdf_list = []

    for file in iso_files:
        gdf = gpd.read_file(file, driver="GeoJSON")
        iso_gdf_list.append(gdf)
    concat_iso = gpd.GeoDataFrame(pd.concat(iso_gdf_list, ignore_index=True))
    concat_iso = concat_iso.set_crs("epsg:4326")
    return concat_iso

In [7]:
hospitals_iso = concat_health_contours(hospital_files)
brgy_healthcenters_iso = concat_health_contours(brgy_healthcenter_files)
rhu_iso = concat_health_contours(rhu_files)

In [8]:
# preview table
rhu_iso

,contour,uid,geometry
0,22,26.0,"POLYGON ((122.07510 6.95991, 122.07089 6.95955..."
1,21,26.0,"POLYGON ((122.07510 6.95811, 122.07044 6.95599..."
2,20,26.0,"POLYGON ((122.07510 6.95544, 122.07309 6.95313..."
3,22,31.0,"POLYGON ((122.09516 6.95940, 122.09316 6.96080..."
4,21,31.0,"POLYGON ((122.09316 6.95831, 122.08831 6.95533..."
...,...,...,...
3089,9,20505.0,"POLYGON ((125.61269 7.08282, 125.61232 7.08088..."
3090,8,20505.0,"POLYGON ((125.61269 7.07991, 125.61169 7.07815..."
3091,10,20824.0,"POLYGON ((125.60323 7.13686, 125.60123 7.13686..."
3092,9,20824.0,"POLYGON ((125.60523 7.13590, 125.60223 7.13574..."


## Load AOI

In [9]:
aoi = gpd.read_file(ADMIN_BOUNDS)  # barangays shapefile
aoi.head(3)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060..."


## Generate grids

In [10]:
bing_tile_grid_generator = grids.BingTileGridGenerator(ZOOM_LEVEL)

In [ ]:
%%time
bing_tile_gdf = bing_tile_grid_generator.generate_grid(aoi)

CPU times: user 9min 49s, sys: 255 ms, total: 9min 49s
Wall time: 9min 49s


In [ ]:
bing_tile_gdf.shape

(223316, 2)

### Add admin boundary columns to grids

In [ ]:
bing_tile_gdf = shi.get_highest_intersection(bing_tile_gdf, aoi, LOCAL_CRS)
bing_tile_gdf.head(2)

### Add population to the grids

In [ ]:
bing_tile_gdf = rzs.create_raster_zonal_stats(
    bing_tile_gdf,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="population_count"),
    extra_args=dict(nodata=-99999),
)

In [ ]:
bing_tile_gdf.head(3)

,geometry,quadkey,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,population_count
0,"POLYGON ((121.00616 14.36950, 121.00616 14.370...",132303122020330010,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Poblacion,PH137603005,59.810890
1,"POLYGON ((121.00616 14.36817, 121.00616 14.369...",132303122020330012,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Poblacion,PH137603005,73.748970
2,"POLYGON ((121.00616 14.36684, 121.00616 14.368...",132303122020330030,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Tunasan,PH137603008,38.532951


In [ ]:
bing_tile_gdf.shape

(223316, 11)

## Convert contours to grids

In [ ]:
def get_min_travel_time(health_iso, facility_type):
    iso_grids = vzs.create_zonal_stats(
        bing_tile_gdf,
        health_iso,
        aggregations=[
            {
                "func": "min",
                "column": "contour",
                "output": f"{facility_type}_travel_time",
            }
        ],
    )
    return iso_grids

In [ ]:
%%time

hospital_grids = get_min_travel_time(hospitals_iso, "hospital")
brgy_healthcenter_grids = get_min_travel_time(
    brgy_healthcenters_iso, "brgy_healthcenters"
)
rhu_grids = get_min_travel_time(rhu_iso, "rhu")

CPU times: user 8min 41s, sys: 3.61 s, total: 8min 45s
Wall time: 8min 45s


In [ ]:
hospital_grids.head(2)

,geometry,quadkey,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,population_count,hospital_travel_time
0,"POLYGON ((121.00616 14.36950, 121.00616 14.370...",132303122020330010,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Poblacion,PH137603005,59.81089,15.0
1,"POLYGON ((121.00616 14.36817, 121.00616 14.369...",132303122020330012,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Poblacion,PH137603005,73.74897,15.0


In [ ]:
# remove geom columns
hospital_grids = hospital_grids.drop(columns=["geometry"])
brgy_healthcenter_grids = brgy_healthcenter_grids.drop(columns=["geometry"])
rhu_grids = rhu_grids.drop(columns=["geometry"])

In [ ]:
# save these
hospital_grids.to_csv(PROCESSED_DIR / "hospital_grids_wadm.csv", index=False)
brgy_healthcenter_grids.to_csv(
    PROCESSED_DIR / "brgy_healthcenter_grids_wadm.csv", index=False
)
rhu_grids.to_csv(PROCESSED_DIR / "rhu_grids_wadm.csv", index=False)

In [11]:
hospital_grids = pd.read_csv(
    PROCESSED_DIR / "hospital_grids_wadm.csv", dtype={"quadkey": str}
)
brgy_healthcenter_grids = pd.read_csv(
    PROCESSED_DIR / "brgy_healthcenter_grids_wadm.csv", dtype={"quadkey": str}
)
rhu_grids = pd.read_csv(PROCESSED_DIR / "rhu_grids_wadm.csv", dtype={"quadkey": str})

## Get barangay-level population

In [13]:
%%time

# first to AOI to get total population
aoi_population_brgy = rzs.create_raster_zonal_stats(
    aoi,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="total_population_count"),
    extra_args=dict(nodata=-99999),
)

CPU times: user 5.49 s, sys: 206 ms, total: 5.69 s
Wall time: 6.31 s


In [14]:
aoi_population_brgy.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,total_population_count
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1048.389526
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",8001.150879


## Get city-level population

In [15]:
city_aoi = aoi.dissolve("ADM3_PCODE")
city_aoi = city_aoi.reset_index()
city_aoi = city_aoi[
    [
        "ADM1_EN",
        "ADM1_PCODE",
        "ADM2_EN",
        "ADM2_PCODE",
        "ADM3_EN",
        "ADM3_PCODE",
        "geometry",
    ]
]
city_aoi

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,"POLYGON ((120.34555 16.02122, 120.34569 16.020..."
1,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,"POLYGON ((121.09935 15.46091, 121.09917 15.461..."
2,Region V,PH050000000,Albay,PH050500000,Legazpi City,PH050506000,"POLYGON ((123.71838 13.06477, 123.71770 13.067..."
3,Region VI,PH060000000,Iloilo,PH063000000,Iloilo City,PH063022000,"POLYGON ((122.54390 10.68390, 122.54387 10.683..."
4,Region VII,PH070000000,Cebu,PH072200000,Mandaue City,PH072230000,"POLYGON ((123.93287 10.31535, 123.93228 10.314..."
5,Region VIII,PH080000000,Leyte,PH083700000,Tacloban City,PH083747000,"MULTIPOLYGON (((124.99348 11.19144, 124.99178 ..."
6,Region IX,PH090000000,Zamboanga del Sur,PH097300000,Zamboanga City,PH097332000,"MULTIPOLYGON (((122.05855 6.87256, 122.05883 6..."
7,Region X,PH100000000,Misamis Oriental,PH104300000,Cagayan de Oro City,PH104305000,"MULTIPOLYGON (((124.57778 8.17860, 124.57721 8..."
8,Region XI,PH110000000,Davao del Sur,PH112400000,Davao City,PH112402000,"POLYGON ((125.40651 6.96856, 125.40641 6.96856..."
9,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,"POLYGON ((121.03750 14.56745, 121.03678 14.567..."


In [16]:
%%time

# first to AOI to get total population
aoi_population_city = rzs.create_raster_zonal_stats(
    city_aoi,
    WP_2020_RASTER,
    aggregation=dict(func="sum", column="population", output="total_population_count"),
    extra_args=dict(nodata=-99999),
)
aoi_population_city.head(2)

CPU times: user 1.17 s, sys: 59.7 ms, total: 1.23 s
Wall time: 1.27 s


,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,geometry,total_population_count
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,"POLYGON ((120.34555 16.02122, 120.34569 16.020...",208321.781250
1,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,"POLYGON ((121.09935 15.46091, 121.09917 15.461...",55563.117188


## Get population reached for each travel time


In [17]:
def compute_population_reached(
    health_iso_grids,
    facility_type,
    admin_level_pcode="ADM3_PCODE",
    aoi_pop_df=aoi_population_city,
):

    aoi_population_copy = aoi_pop_df.copy()
    health_iso_grids_copy = health_iso_grids.copy()

    # get travel time column
    suffix = "_travel_time"
    selected_col = [
        col for col in health_iso_grids_copy.columns if col.endswith(suffix)
    ]
    selected_col = selected_col[0]

    health_iso_grids_copy = health_iso_grids_copy.sort_values(
        by=[f"{facility_type}_travel_time"]
    )

    # group by admin level and travel time
    agg_cumsum_admin_pop = (
        health_iso_grids_copy.groupby(
            [admin_level_pcode, f"{facility_type}_travel_time"], dropna=False
        )["population_count"]
        .sum()
        .reset_index(name="population_count_reached")
    )

    agg_cumsum_admin_pop["pop_count_cumsum"] = agg_cumsum_admin_pop.groupby(
        admin_level_pcode, dropna=False
    )["population_count_reached"].cumsum()
    agg_cumsum_admin_pop = agg_cumsum_admin_pop.dropna(
        subset=f"{facility_type}_travel_time"
    )

    merged_total = aoi_population_copy.merge(
        agg_cumsum_admin_pop, on=admin_level_pcode, how="left"
    )
    merged_total = merged_total.drop_duplicates(
        subset=[admin_level_pcode, f"{facility_type}_travel_time"]
    )
    # compute percent population reached
    merged_total["pct_population_reached"] = (
        merged_total["pop_count_cumsum"] / merged_total["total_population_count"]
    ) * 100
    # round numbers
    merged_total["pop_count_cumsum"] = merged_total["pop_count_cumsum"].round(2)
    merged_total["total_population_count"] = merged_total[
        "total_population_count"
    ].round(2)
    merged_total["pct_population_reached"] = merged_total[
        "pct_population_reached"
    ].round(2)

    # for rows with greater than 100%
    merged_total.loc[
        (merged_total["pct_population_reached"] > 100), "pct_population_reached"
    ] = 100.00
    merged_total.loc[
        (merged_total["pct_population_reached"] == 100), "pop_count_cumsum"
    ] = merged_total.loc[
        (merged_total["pct_population_reached"] == 100), "total_population_count"
    ]

    merged_total = merged_total.drop(columns=["geometry", "population_count_reached"])
    return merged_total

In [18]:
def fill_time(pop_reached_df, facility_type="hospital", admin_level_pcode="ADM3_PCODE"):
    df = pop_reached_df.copy()

    all_times = pd.Series(range(5, 31), name=f"{facility_type}_travel_time")
    all_pcodes = df[admin_level_pcode].unique()
    all_combinations = pd.DataFrame(
        [(x, y) for x in all_pcodes for y in all_times],
        columns=[admin_level_pcode, f"{facility_type}_travel_time"],
    )

    # set cols list based on admin level code groupby
    if admin_level_pcode == "ADM3_PCODE":
        select_df_cols = [
            "ADM1_EN",
            "ADM1_PCODE",
            "ADM2_EN",
            "ADM2_PCODE",
            "ADM3_EN",
            "ADM3_PCODE",
            "total_population_count",
        ]
        cols_order = [
            "ADM1_EN",
            "ADM1_PCODE",
            "ADM2_EN",
            "ADM2_PCODE",
            "ADM3_EN",
            "ADM3_PCODE",
            f"{facility_type}_travel_time",
            "total_population_count",
            "pop_count_cumsum",
            "pct_population_reached",
        ]
    elif admin_level_pcode == "ADM4_PCODE":
        select_df_cols = [
            "ADM1_EN",
            "ADM1_PCODE",
            "ADM2_EN",
            "ADM2_PCODE",
            "ADM3_EN",
            "ADM3_PCODE",
            "ADM4_EN",
            "ADM4_PCODE",
            "total_population_count",
        ]
        cols_order = [
            "ADM1_EN",
            "ADM1_PCODE",
            "ADM2_EN",
            "ADM2_PCODE",
            "ADM3_EN",
            "ADM3_PCODE",
            "ADM4_EN",
            "ADM4_PCODE",
            f"{facility_type}_travel_time",
            "total_population_count",
            "pop_count_cumsum",
            "pct_population_reached",
        ]

    # Merge the original dataset with the DataFrame of all combinations
    # add aoi columns first and admin level total population count
    filled_df = pd.merge(
        all_combinations,
        df[select_df_cols],
        on=[admin_level_pcode],  # , f'{facility_type}_travel_time'
        how="left",
    )
    filled_df = filled_df.drop_duplicates(
        subset=[admin_level_pcode, f"{facility_type}_travel_time"]
    )

    # add pop count cumulative sum
    filled_df = pd.merge(
        filled_df,
        df[
            [
                admin_level_pcode,
                f"{facility_type}_travel_time",
                "pop_count_cumsum",
                "pct_population_reached",
            ]
        ],
        on=[admin_level_pcode, f"{facility_type}_travel_time"],
        how="left",
    )
    filled_df = filled_df.dropna(subset=[admin_level_pcode])

    # add a groupby for city
    filled_df = filled_df.groupby([admin_level_pcode], group_keys=False).apply(
        lambda x: x.fillna(method="ffill")
    )
    filled_df = filled_df.fillna(0)

    # reorganize columns
    filled_df = filled_df[cols_order]

    return filled_df

In [33]:
hospitals_pop_reached = compute_population_reached(
    hospital_grids,
    "hospital",
    admin_level_pcode="ADM4_PCODE",
    aoi_pop_df=aoi_population_brgy,
)

In [34]:
final_hospitals_pop_reached = fill_time(
    hospitals_pop_reached, facility_type="hospital", admin_level_pcode="ADM4_PCODE"
)
final_hospitals_pop_reached.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,hospital_travel_time,total_population_count,pop_count_cumsum,pct_population_reached
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,5,1048.39,0.0,0.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,6,1048.39,0.0,0.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,7,1048.39,0.0,0.0
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,8,1048.39,0.0,0.0
4,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,9,1048.39,0.0,0.0


In [23]:
final_hospitals_pop_reached["ADM3_EN"].unique()

array(['Dagupan City', 'Palayan City', 'Legazpi City', 'Iloilo City',
       'Mandaue City', 'Tacloban City', 'Zamboanga City',
       'Cagayan de Oro City', 'Davao City', 'City of Mandaluyong',
       'City of Navotas', 'City of Muntinlupa'], dtype=object)

In [42]:
final_hospitals_pop_reached

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,hospital_travel_time,total_population_count,pop_count_cumsum,pct_population_reached
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,5,1048.39,0.00,0.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,6,1048.39,0.00,0.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,7,1048.39,0.00,0.0
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,8,1048.39,0.00,0.0
4,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,9,1048.39,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22849,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,"North Bay Blvd., North",PH137503008,26,11461.58,11461.58,100.0
22850,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,"North Bay Blvd., North",PH137503008,27,11461.58,11461.58,100.0
22851,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,"North Bay Blvd., North",PH137503008,28,11461.58,11461.58,100.0
22852,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,"North Bay Blvd., North",PH137503008,29,11461.58,11461.58,100.0


In [36]:
brgy_healthcenter_pop_reached = compute_population_reached(
    brgy_healthcenter_grids,
    "brgy_healthcenters",
    admin_level_pcode="ADM4_PCODE",
    aoi_pop_df=aoi_population_brgy,
)

In [37]:
final_healthcenter_pop_reached = fill_time(
    brgy_healthcenter_pop_reached,
    facility_type="brgy_healthcenters",
    admin_level_pcode="ADM4_PCODE",
)
final_healthcenter_pop_reached.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,brgy_healthcenters_travel_time,total_population_count,pop_count_cumsum,pct_population_reached
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,5,1048.39,0.0,0.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,6,1048.39,0.0,0.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,7,1048.39,0.0,0.0
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,8,1048.39,0.0,0.0
4,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,9,1048.39,0.0,0.0


In [38]:
rhu_pop_reached = compute_population_reached(
    rhu_grids, "rhu", aoi_pop_df=aoi_population_brgy, admin_level_pcode="ADM4_PCODE"
)

In [39]:
final_rhu_pop_reached = fill_time(
    rhu_pop_reached, facility_type="rhu", admin_level_pcode="ADM4_PCODE"
)
final_rhu_pop_reached.head()

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,rhu_travel_time,total_population_count,pop_count_cumsum,pct_population_reached
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,5,1048.39,0.0,0.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,6,1048.39,0.0,0.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,7,1048.39,0.0,0.0
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,8,1048.39,0.0,0.0
4,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,9,1048.39,0.0,0.0


In [28]:
final_rhu_pop_reached["ADM3_EN"].unique()

array(['Dagupan City', 'Palayan City', 'Legazpi City', 'Iloilo City',
       'Mandaue City', 'Tacloban City', 'Zamboanga City',
       'Cagayan de Oro City', 'Davao City', 'City of Mandaluyong',
       'City of Navotas', 'City of Muntinlupa'], dtype=object)

## Save and export files

In [47]:
ADM_LEVEL = "brgy"

In [49]:
filtered_hospitals = final_hospitals_pop_reached.copy()
filtered_hospitals = filtered_hospitals[
    (filtered_hospitals["hospital_travel_time"] == 5)
    | (filtered_hospitals["hospital_travel_time"] == 15)
    | (filtered_hospitals["hospital_travel_time"] == 30)
].reset_index(drop=True)
filtered_hospitals.to_csv(
    OUTPUT_DIR / f"hospital_isochrones/hospitals_pop_reached_{ADM_LEVEL}level.csv",
    index=False,
)

In [50]:
filtered_hc = final_healthcenter_pop_reached.copy()
filtered_hc = filtered_hc[
    (filtered_hc["brgy_healthcenters_travel_time"] == 5)
    | (filtered_hc["brgy_healthcenters_travel_time"] == 15)
    | (filtered_hc["brgy_healthcenters_travel_time"] == 30)
].reset_index(drop=True)
filtered_hc.to_csv(
    OUTPUT_DIR
    / f"hospital_isochrones/brgy_healthcenter_pop_reached_{ADM_LEVEL}level.csv",
    index=False,
)

In [51]:
filtered_rhu = final_rhu_pop_reached.copy()
filtered_rhu = filtered_rhu[
    (filtered_rhu["rhu_travel_time"] == 5)
    | (filtered_rhu["rhu_travel_time"] == 15)
    | (filtered_rhu["rhu_travel_time"] == 30)
].reset_index(drop=True)
filtered_rhu.to_csv(
    OUTPUT_DIR / f"hospital_isochrones/rhu_pop_reached_{ADM_LEVEL}level.csv",
    index=False,
)

In [52]:
final_hospitals_pop_reached.to_csv(
    PROCESSED_DIR / f"revised_hospitals_pop_reached_{ADM_LEVEL}level.csv", index=False
)
final_healthcenter_pop_reached.to_csv(
    PROCESSED_DIR / f"revised_brgy_healthcenter_pop_reached_{ADM_LEVEL}level.csv",
    index=False,
)
final_rhu_pop_reached.to_csv(
    PROCESSED_DIR / f"revised_rhu_pop_reached_{ADM_LEVEL}level.csv", index=False
)